In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from delf import delf_config_pb2, extractor, utils
from google.protobuf import text_format
from scipy import spatial
import pydegensac
from skimage.feature import plot_matches

In [ ]:
DELG_CONFIG = "/opt/models/research/delf/delf/python/delg/r101delg_gldv2clean_config.pbtxt"

In [ ]:
config = delf_config_pb2.DelfConfig()

with tf.io.gfile.GFile(DELG_CONFIG, "r") as f:
    text_format.Parse(f.read(), config)

extractor_fn = extractor.MakeExtractor(config)

In [ ]:
filepath_list = [
    "../data/gldv2_micro/images/457cb65ba4a1ee3d.jpg",
    "../data/gldv2_micro/images/1382256e230d5696.jpg",
]
    
results = {}
for filepath in filepath_list:
    resize_factor = 1.0
    img = np.array(utils.RgbLoader(filepath))
    
    extract_features = extractor_fn(img, resize_factor)
    
    global_descriptor = extract_features["global_descriptor"]
    local_features = extract_features["local_features"]
    
    locations = local_features["locations"]
    descriptors = local_features["descriptors"]
    scales = local_features["scales"]
    attention = local_features["attention"]
    
    results[filepath] = {
        "locations" : locations.astype(np.int16),
        "descriptors": descriptors,
        "img": img
    }

In [ ]:
for filepath in filepath_list:
    locations = results[filepath]["locations"]
    img = results[filepath]["img"]
    plt.imshow(img)
    plt.scatter(locations[:, 1], locations[:, 0])
    plt.show()

In [ ]:
filepath1 = filepath_list[0]
locations1 = results[filepath1]["locations"]
descriptors1 = results[filepath1]["descriptors"]
img1 = results[filepath1]["img"]

In [ ]:
filepath2 = filepath_list[1]
locations2 = results[filepath2]["locations"]
descriptors2 = results[filepath2]["descriptors"]
img2 = results[filepath2]["img"]

In [ ]:
locations1.shape, locations2.shape, descriptors1.shape, descriptors2.shape

In [ ]:
d1_tree = spatial.cKDTree(descriptors1)
_, indices = d1_tree.query(descriptors2, distance_upper_bound=0.8)
indices.shape

In [ ]:
pts2 = np.array([locations2[i, :] for i in range(locations2.shape[0]) if indices[i] != locations2.shape[0]])
pts1 = np.array([locations1[indices[i], :] for i in range(locations1.shape[0]) if indices[i] != locations1.shape[0]])

In [ ]:
plt.imshow(img1)
plt.scatter(pts1[:, 1], pts1[:, 0])
plt.show()
plt.imshow(img2)
plt.scatter(pts2[:, 1], pts2[:, 0])
plt.show()

In [ ]:
threshold = 4.0
conf = 0.99
n_iters = 2000
_, mask = pydegensac.findFundamentalMatrix(pts1, pts2, threshold, conf, n_iters)

In [ ]:
sum(mask)

In [ ]:
inlier_idxs = np.nonzero(mask)[0]

In [ ]:
inlier_idxs

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 5), dpi=300)
plot_matches(
    ax,
    img1,
    img2,
    pts1,
    pts2,
    np.column_stack((inlier_idxs, inlier_idxs)),
    matches_color='r')
plt.show()